In [36]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
import spacy
import re
import spacy.lang.ru
from spacy.lang.ru.stop_words import STOP_WORDS
from spacy.cli import download
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pymorphy2
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
import transformers
from transformers import BertModel
from transformers.models.bert.tokenization_bert import BertTokenizer
from pathlib import Path
from nltk.corpus import stopwords
import torch

In [37]:
def dis(dataframe):
    display(dataframe.shape, dataframe.head(4), dataframe.tail(4))
    return None

# функция для нужного нам округления с трешхолдом
def my_round(x, tr_hold=0.5):
    if x >= tr_hold:
        return 1
    if x <= -tr_hold:
        return -1
    return 0

#TF_IDF + lemmatization
def punctuation(line):
    return re.sub(r'[.,!?\'\"]', '', line)

def removing_stopwords(line):
    text_tokens = word_tokenize(line)
    remove_sw = [word for word in text_tokens if not word in stopwords.words()]
    res = ''
    for i in range(len(remove_sw)):
        res += ''.join(remove_sw[i]) + ' '
    return res[:-1]

def extract_keywords(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([i for i in text if not i.isdigit()])
    text = ' '.join(text.strip().split())
    doc = nlp(text)
    stop_words = spacy.lang.ru.stop_words.STOP_WORDS
    date_names = set(['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь',
                  'понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье'])
    for word in date_names:
        stop_words.add(word.lower())
    lemmas = [token.lemma_ for token in doc if ((not token.is_stop) and token.lemma_ not in stop_words)]
    lemmi = list(set(lemmas))
    res =''
    for i in range(0, len(lemmi)):
        res += ''.join(lemmi[i]) + ' '
    return res[:-1]

# функция для удаления знаков препинания
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

morph = pymorphy2.MorphAnalyzer()
def lemmatization(line):
    text_tokens = word_tokenize(line)
    remove_sw = [word for word in text_tokens if not word in stopwords.words()]
    res = ''
    for i in range(0, len(remove_sw)):
        elem = morph.parse(remove_sw[i])[0][2]
        res += ''.join(elem) + ' '
    return res[:-1]

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
df = pd.read_csv('/home/jupyter/mnt/s3/hsedatafitpredict1392/new_test.csv', index_col=0)
df

,texts
4036,15.03.2022 обратился на горячую линию для закр...
5804,"Уже который год в ТКБ не решается ""глобальная ..."
2752,Добрый день. Хочу оставить отзыв о пользовании...
1921,"Добрый день Сегодня, зайдя в свой личный кабин..."
7374,"Обслуживаюсь в Тинькофф пару лет, возникла жес..."
...,...
146,Отвратительный сервис и отношение к клиентам! ...
2677,28.04.2022 обратилась в банк о возможности пер...
4481,В начале 2021 года была акция по выплате 8% ке...
4112,Бездействие банка и некомпетентность сотрудник...


In [49]:
df['texts'].iloc[2]

'Добрый день. Хочу оставить отзыв о пользовании дебетовой карты Тинькофф Блэк и о Тинькофф банк приложении. Пользуюсь данной картой уже длительное время, около 4 лет. Считаю ее одним из лучших предложений на рынке.  Расскажу о плюсах. Начну с...'

In [39]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
#!g1.1
df['without_punct'] = df['texts'].apply(punctuation)

In [52]:
#!c1.8
df['without_stopwords'] = df['without_punct'].apply(removing_stopwords)

In [53]:
#!c1.8
df['lemmatized'] = df['without_stopwords'].apply(lemmatization)

In [54]:
#!c1.8
df

,texts,without_punct,without_stopwords,lemmatized
4036,15.03.2022 обратился на горячую линию для закр...,15032022 обратился на горячую линию для закрыт...,15032022 обратился горячую линию закрытия счет...,15032022 обратиться горячий линия закрытие счё...
5804,"Уже который год в ТКБ не решается ""глобальная ...",Уже который год в ТКБ не решается глобальная п...,Уже который год ТКБ решается глобальная пробле...,уже который год ткб решаться глобальный пробле...
2752,Добрый день. Хочу оставить отзыв о пользовании...,Добрый день Хочу оставить отзыв о пользовании ...,Добрый день Хочу оставить отзыв пользовании де...,добрый день хотеть оставить отзыв пользование ...
1921,"Добрый день Сегодня, зайдя в свой личный кабин...",Добрый день Сегодня зайдя в свой личный кабине...,Добрый день Сегодня зайдя свой личный кабинет ...,добрый день сегодня зайти свой личный кабинет ...
7374,"Обслуживаюсь в Тинькофф пару лет, возникла жес...",Обслуживаюсь в Тинькофф пару лет возникла жест...,Обслуживаюсь Тинькофф пару лет возникла жесточ...,обслуживаться тинькофф пара год возникнуть жес...
...,...,...,...,...
146,Отвратительный сервис и отношение к клиентам! ...,Отвратительный сервис и отношение к клиентам Я...,Отвратительный сервис отношение клиентам Являю...,отвратительный сервис отношение клиент являтьс...
2677,28.04.2022 обратилась в банк о возможности пер...,28042022 обратилась в банк о возможности перев...,28042022 обратилась банк возможности перевода ...,28042022 обратиться банк возможность перевод 1...
4481,В начале 2021 года была акция по выплате 8% ке...,В начале 2021 года была акция по выплате 8% ке...,В начале 2021 года акция выплате 8 % кешбека о...,в начало 2021 год акция выплата 8 % кешбек опл...
4112,Бездействие банка и некомпетентность сотрудник...,Бездействие банка и некомпетентность сотрудник...,Бездействие банка некомпетентность сотрудников...,бездействие банк некомпетентность сотрудник 10...


In [55]:
#!c1.8
#!c1.32
# Определение корпуса текстов
corpus = df['texts'].tolist()
# Инициализация векторизатора
vectorizer = TfidfVectorizer()

In [56]:
#!c1.8
#!c1.32
# Преобразование текстов в векторы TF-IDF
vectors = vectorizer.fit_transform(corpus)

# Получение списка слов в порядке их индексации
words = vectorizer.get_feature_names()

In [57]:
#!c1.8
#!c1.32
# Создание DataFrame с векторами TF-IDF
tfidf_df_test = pd.DataFrame(vectors.toarray(), columns=words)
tfidf_df_test

,00,000,0000,000р,00600140,00руб,01,02,0200,0216,03,0352,04,05,0503,056300,06,06105,07,0708,0776,08,0811,09,0981,10,100,1000,10000,100000,100000р,10000р,1000р,1005,101а,10548884,10647745,108,10к,11,...,этому,этот,эту,эффекта,эффективно,юmoney,южной,юл,юлии,юмани,юнистрим,юр,юридический,юридическими,юридического,юрист,юрлицо,явился,являемся,является,являлась,являлся,являюсь,являющимся,являясь,явный,языке,якобы,ян,январе,января,яндекс,яндекса,япфр,яркое,яровое,яровом,ясна,яснить,ячейки
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.155330,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.26244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.149388,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
#!c1.8
tfidf_df_test.to_csv('/home/jupyter/mnt/s3/hsedatafitpredict1392/tfidf_test.csv')

In [ ]:
#!c1.8
